# import libraries

In [1]:
import mysql.connector
import pandas as pd

# Extract

In [2]:
# function to extract data
def extract_data(host, user, password, database, table_names):
    # Create a connection
    connection = mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )

    # Initialize an empty dictionary to store DataFrames
    extracted_data = {}

    # Loop through the table names and extract data
    for table_name in table_names:
        sql_query = "SELECT * FROM {}".format(table_name)
        df = pd.read_sql_query(sql_query, connection)
        
        # Store the DataFrame in the dictionary
        extracted_data[table_name] = df

        # Print the first few rows of each DataFrame
        print("Data from table '{}':".format(table_name))
        print(df.head())
        print("\n")

    # Close the database connection
    connection.close()

    return extracted_data

# database information
host = "localhost"
user = "root"
password = ""
database = "bike_store"
table_names = ["brands", "categories", "customers","products","stores","stocks","orders", "order_items","staffs"]

extracted_data = extract_data(host, user, password, database, table_names)

Data from table 'brands':
   brand_id   brand_name
0         1      Electra
1         2         Haro
2         3       Heller
3         4  Pure Cycles
4         5      Ritchey


Data from table 'categories':
   category_id        category_name
0            1    Children Bicycles
1            2     Comfort Bicycles
2            3    Cruisers Bicycles
3            4  Cyclocross Bicycles
4            5       Electric Bikes


Data from table 'customers':
   customer_id  first_name last_name           phone                    email  \
0            1       Debra     Burks            None    debra.burks@yahoo.com   
1            2       Kasha      Todd            None     kasha.todd@yahoo.com   
2            3      Tameka    Fisher            None    tameka.fisher@aol.com   
3            4       Daryl    Spence            None     daryl.spence@aol.com   
4            5  Charolette      Rice  (916) 381-6003  charolette.rice@msn.com   

                  street           city state  zip_code  


In [3]:
# Convert date columns to datetime
date_columns = ['order_date', 'required_date', 'shipped_date']
for col in date_columns:
    extracted_data['orders'][col] = pd.to_datetime(extracted_data['orders'][col])

# Extract year, month, and day into separate columns
for col in date_columns:
    extracted_data['orders']['{}_year'.format(col)] = extracted_data['orders'][col].dt.year
    extracted_data['orders']['{}_month'.format(col)] = extracted_data['orders'][col].dt.month
    extracted_data['orders']['{}_day'.format(col)] = extracted_data['orders'][col].dt.day

# Drop the original date columns
extracted_data['orders'].drop(columns=date_columns, inplace=True)

# Fill NaN values with 0 and convert to integers
date_columns_with_prefix = ['{}_year'.format(col) for col in date_columns] + ['{}_month'.format(col) for col in date_columns] + ['{}_day'.format(col) for col in date_columns]
extracted_data['orders'][date_columns_with_prefix] = extracted_data['orders'][date_columns_with_prefix].fillna(0).astype(int)

# Drop multiple columns from customers
columns_to_drop = ['first_name', 'last_name', 'phone', 'email', 'zip_code']
extracted_data['customers'].drop(columns=columns_to_drop, inplace=True)
# Drop multiple columns from stores
columns_to_drop = ['phone', 'email', 'zip_code']
extracted_data['stores'].drop(columns=columns_to_drop, inplace=True)


# Drop multiple columns from orders
columns_to_drop = ['order_status','staff_id']
extracted_data['orders'].drop(columns=columns_to_drop, inplace=True)

# Drop multiple columns from orders
columns_to_drop = ['item_id']
extracted_data['order_items'].drop(columns=columns_to_drop, inplace=True)

extracted_data.pop('staffs', None)
table_names = ["brands", "categories", "customers","products","stores","stocks","orders", "order_items"]

extracted_data = extract_data(host, user, password, database, table_names)

Data from table 'brands':
   brand_id   brand_name
0         1      Electra
1         2         Haro
2         3       Heller
3         4  Pure Cycles
4         5      Ritchey


Data from table 'categories':
   category_id        category_name
0            1    Children Bicycles
1            2     Comfort Bicycles
2            3    Cruisers Bicycles
3            4  Cyclocross Bicycles
4            5       Electric Bikes


Data from table 'customers':
   customer_id  first_name last_name           phone                    email  \
0            1       Debra     Burks            None    debra.burks@yahoo.com   
1            2       Kasha      Todd            None     kasha.todd@yahoo.com   
2            3      Tameka    Fisher            None    tameka.fisher@aol.com   
3            4       Daryl    Spence            None     daryl.spence@aol.com   
4            5  Charolette      Rice  (916) 381-6003  charolette.rice@msn.com   

                  street           city state  zip_code  


In [4]:
# Database connection
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="dw_bike_store"
)

# Create a cursor object to execute SQL queries
cursor = connection.cursor()
try:
    # Iterate through the tables in extracted_data and insert the data
    for table_name, df in extracted_data.items():
        if not df.empty:
            # Build the INSERT query dynamically
            columns = ', '.join(df.columns)
            placeholders = ', '.join(['%s'] * len(df.columns))
            # Build the INSERT query dynamically with ON DUPLICATE KEY UPDATE
            query = "INSERT INTO {} ({}) VALUES ({}) ON DUPLICATE KEY UPDATE {}".format(
                table_name, columns, placeholders, ', '.join(["{}=VALUES({})".format(col, col) for col in df.columns])
            )

            # Insert the data into the database
            values = [tuple(str(val) for val in row) for row in df.values]

            for i, original_row in enumerate(values):
                try:
                    cursor.execute(query, original_row)
                except mysql.connector.Error as row_err:
                    print("Error in row {}:".format(i + 1))
                    for col, val in zip(df.columns, original_row):
                        print(col, ":", val)
                    print("Error:", row_err)

            connection.commit()

    # Calculate total sales by customer and insert into retail_facts
    total_sale_by_customer_query = """
    INSERT INTO retail_facts (customer_id, total_sale_by_customer)
    SELECT c.customer_id, SUM(oi.list_price) as total_sale_by_customer
    FROM customers AS c
    JOIN orders AS o ON c.customer_id = o.customer_id
    JOIN order_items AS oi ON o.order_id = oi.order_id
    GROUP BY c.customer_id
    ON DUPLICATE KEY UPDATE total_sale_by_customer = VALUES(total_sale_by_customer)
    """

    cursor.execute(total_sale_by_customer_query)

    # Calculate purchase frequency and insert into retail_facts
    purchase_frequency_query = """
    INSERT INTO retail_facts (customer_id, purchase_frequency)
    SELECT c.customer_id, COUNT(o.order_id) as purchase_frequency
    FROM customers AS c
    JOIN orders AS o ON c.customer_id = o.customer_id
    GROUP BY c.customer_id
    ON DUPLICATE KEY UPDATE purchase_frequency = VALUES(purchase_frequency)
    """

    cursor.execute(purchase_frequency_query)

    # Calculate the number of products purchased by customer and insert into retail_facts
    number_products_by_customer_query = """
    INSERT INTO retail_facts (customer_id, number_products_by_customer)
    SELECT c.customer_id, COUNT(DISTINCT oi.product_id) as number_products_by_customer
    FROM customers AS c
    JOIN orders AS o ON c.customer_id = o.customer_id
    JOIN order_items AS oi ON o.order_id = oi.order_id
    GROUP BY c.customer_id
    ON DUPLICATE KEY UPDATE number_products_by_customer = VALUES(number_products_by_customer)
    """

    cursor.execute(number_products_by_customer_query)

    connection.commit()

except mysql.connector.Error as err:
    print(table_name)
    print("Error: {}".format(err))
    connection.rollback()  # Rollback the transaction in case of an error

finally:
    # Close the cursor and connection
    cursor.close()
    connection.close()


Error in row 1:
order_id : 1
customer_id : 259
order_status : 4
order_date : 2021-01-01
required_date : 2016-01-03
shipped_date : 2016-01-03
store_id : 1
staff_id : 2
Error: 1054 (42S22): Unknown column 'order_status' in 'field list'
Error in row 2:
order_id : 2
customer_id : 1212
order_status : 4
order_date : 2021-01-01
required_date : 2016-01-04
shipped_date : 2016-01-03
store_id : 2
staff_id : 6
Error: 1054 (42S22): Unknown column 'order_status' in 'field list'
Error in row 3:
order_id : 3
customer_id : 523
order_status : 4
order_date : 2021-01-02
required_date : 2016-01-05
shipped_date : 2016-01-03
store_id : 2
staff_id : 7
Error: 1054 (42S22): Unknown column 'order_status' in 'field list'
Error in row 4:
order_id : 4
customer_id : 175
order_status : 4
order_date : 2021-01-03
required_date : 2016-01-04
shipped_date : 2016-01-05
store_id : 1
staff_id : 3
Error: 1054 (42S22): Unknown column 'order_status' in 'field list'
Error in row 5:
order_id : 5
customer_id : 1324
order_status : 

Error in row 79:
order_id : 79
customer_id : 529
order_status : 4
order_date : 2021-02-17
required_date : 2016-02-20
shipped_date : 2016-02-20
store_id : 1
staff_id : 3
Error: 1054 (42S22): Unknown column 'order_status' in 'field list'
Error in row 80:
order_id : 80
customer_id : 397
order_status : 4
order_date : 2021-02-17
required_date : 2016-02-20
shipped_date : 2016-02-19
store_id : 2
staff_id : 6
Error: 1054 (42S22): Unknown column 'order_status' in 'field list'
Error in row 81:
order_id : 81
customer_id : 521
order_status : 4
order_date : 2021-02-17
required_date : 2016-02-19
shipped_date : 2016-02-18
store_id : 2
staff_id : 6
Error: 1054 (42S22): Unknown column 'order_status' in 'field list'
Error in row 82:
order_id : 82
customer_id : 1223
order_status : 4
order_date : 2021-02-18
required_date : 2016-02-21
shipped_date : 2016-02-20
store_id : 2
staff_id : 6
Error: 1054 (42S22): Unknown column 'order_status' in 'field list'
Error in row 83:
order_id : 83
customer_id : 393
order_

Error in row 135:
order_id : 135
customer_id : 1354
order_status : 4
order_date : 2021-03-21
required_date : 2016-03-23
shipped_date : 2016-03-23
store_id : 2
staff_id : 7
Error: 1054 (42S22): Unknown column 'order_status' in 'field list'
Error in row 136:
order_id : 136
customer_id : 1192
order_status : 4
order_date : 2021-03-21
required_date : 2016-03-24
shipped_date : 2016-03-23
store_id : 3
staff_id : 8
Error: 1054 (42S22): Unknown column 'order_status' in 'field list'
Error in row 137:
order_id : 137
customer_id : 370
order_status : 4
order_date : 2021-03-23
required_date : 2016-03-26
shipped_date : 2016-03-25
store_id : 2
staff_id : 7
Error: 1054 (42S22): Unknown column 'order_status' in 'field list'
Error in row 138:
order_id : 138
customer_id : 438
order_status : 4
order_date : 2021-03-23
required_date : 2016-03-24
shipped_date : 2016-03-25
store_id : 2
staff_id : 7
Error: 1054 (42S22): Unknown column 'order_status' in 'field list'
Error in row 139:
order_id : 139
customer_id :

Error in row 195:
order_id : 195
customer_id : 727
order_status : 4
order_date : 2021-04-28
required_date : 2016-05-01
shipped_date : 2016-05-01
store_id : 2
staff_id : 7
Error: 1054 (42S22): Unknown column 'order_status' in 'field list'
Error in row 196:
order_id : 196
customer_id : 1247
order_status : 4
order_date : 2021-04-28
required_date : 2016-04-30
shipped_date : 2016-05-01
store_id : 2
staff_id : 6
Error: 1054 (42S22): Unknown column 'order_status' in 'field list'
Error in row 197:
order_id : 197
customer_id : 127
order_status : 4
order_date : 2021-04-30
required_date : 2016-05-03
shipped_date : 2016-05-03
store_id : 2
staff_id : 7
Error: 1054 (42S22): Unknown column 'order_status' in 'field list'
Error in row 198:
order_id : 198
customer_id : 640
order_status : 4
order_date : 2021-05-01
required_date : 2016-05-04
shipped_date : 2016-05-03
store_id : 2
staff_id : 6
Error: 1054 (42S22): Unknown column 'order_status' in 'field list'
Error in row 199:
order_id : 199
customer_id : 

shipped_date : 2016-06-07
store_id : 2
staff_id : 7
Error: 1054 (42S22): Unknown column 'order_status' in 'field list'
Error in row 257:
order_id : 257
customer_id : 1314
order_status : 4
order_date : 2021-06-05
required_date : 2016-06-06
shipped_date : 2016-06-08
store_id : 1
staff_id : 3
Error: 1054 (42S22): Unknown column 'order_status' in 'field list'
Error in row 258:
order_id : 258
customer_id : 107
order_status : 4
order_date : 2021-06-06
required_date : 2016-06-07
shipped_date : 2016-06-07
store_id : 2
staff_id : 7
Error: 1054 (42S22): Unknown column 'order_status' in 'field list'
Error in row 259:
order_id : 259
customer_id : 413
order_status : 4
order_date : 2021-06-06
required_date : 2016-06-09
shipped_date : 2016-06-09
store_id : 2
staff_id : 7
Error: 1054 (42S22): Unknown column 'order_status' in 'field list'
Error in row 260:
order_id : 260
customer_id : 632
order_status : 4
order_date : 2021-06-06
required_date : 2016-06-08
shipped_date : 2016-06-08
store_id : 2
staff_id

...

Error in row 4368:
order_id : 1499.0
item_id : 4.0
product_id : 2.0
quantity : 1.0
list_price : 749.99
discount : 0.05
Error: 1054 (42S22): Unknown column 'item_id' in 'field list'
Error in row 4369:
order_id : 1499.0
item_id : 5.0
product_id : 171.0
quantity : 2.0
list_price : 3199.99
discount : 0.1
Error: 1054 (42S22): Unknown column 'item_id' in 'field list'
Error in row 4370:
order_id : 1500.0
item_id : 1.0
product_id : 41.0
quantity : 2.0
list_price : 1469.99
discount : 0.07
Error: 1054 (42S22): Unknown column 'item_id' in 'field list'
Error in row 4371:
order_id : 1500.0
item_id : 2.0
product_id : 293.0
quantity : 1.0
list_price : 319.99
discount : 0.07
Error: 1054 (42S22): Unknown column 'item_id' in 'field list'
Error in row 4372:
order_id : 1501.0
item_id : 1.0
product_id : 248.0
quantity : 1.0
list_price : 599.99
discount : 0.05
Error: 1054 (42S22): Unknown column 'item_id' in 'field list'
Error in row 4373:
order_id : 1501.0
item_id : 2.0
product_id : 219.0
quantity : 1.0
li

Error: 1054 (42S22): Unknown column 'item_id' in 'field list'
Error in row 4443:
order_id : 1521.0
item_id : 3.0
product_id : 123.0
quantity : 1.0
list_price : 999.99
discount : 0.1
Error: 1054 (42S22): Unknown column 'item_id' in 'field list'
Error in row 4444:
order_id : 1521.0
item_id : 4.0
product_id : 135.0
quantity : 2.0
list_price : 1499.99
discount : 0.2
Error: 1054 (42S22): Unknown column 'item_id' in 'field list'
Error in row 4445:
order_id : 1522.0
item_id : 1.0
product_id : 95.0
quantity : 1.0
list_price : 299.99
discount : 0.1
Error: 1054 (42S22): Unknown column 'item_id' in 'field list'
Error in row 4446:
order_id : 1522.0
item_id : 2.0
product_id : 141.0
quantity : 1.0
list_price : 1599.99
discount : 0.07
Error: 1054 (42S22): Unknown column 'item_id' in 'field list'
Error in row 4447:
order_id : 1522.0
item_id : 3.0
product_id : 117.0
quantity : 1.0
list_price : 1469.99
discount : 0.07
Error: 1054 (42S22): Unknown column 'item_id' in 'field list'
Error in row 4448:
order

quantity : 2.0
list_price : 469.99
discount : 0.05
Error: 1054 (42S22): Unknown column 'item_id' in 'field list'
Error in row 4491:
order_id : 1539.0
item_id : 2.0
product_id : 12.0
quantity : 2.0
list_price : 549.99
discount : 0.05
Error: 1054 (42S22): Unknown column 'item_id' in 'field list'
Error in row 4492:
order_id : 1540.0
item_id : 1.0
product_id : 71.0
quantity : 2.0
list_price : 416.99
discount : 0.05
Error: 1054 (42S22): Unknown column 'item_id' in 'field list'
Error in row 4493:
order_id : 1540.0
item_id : 2.0
product_id : 236.0
quantity : 1.0
list_price : 749.99
discount : 0.07
Error: 1054 (42S22): Unknown column 'item_id' in 'field list'
Error in row 4494:
order_id : 1540.0
item_id : 3.0
product_id : 224.0
quantity : 1.0
list_price : 479.99
discount : 0.05
Error: 1054 (42S22): Unknown column 'item_id' in 'field list'
Error in row 4495:
order_id : 1540.0
item_id : 4.0
product_id : 177.0
quantity : 2.0
list_price : 5499.99
discount : 0.05
Error: 1054 (42S22): Unknown column

Error in row 4596:
order_id : 1573.0
item_id : 5.0
product_id : 263.0
quantity : 2.0
list_price : 89.99
discount : 0.1
Error: 1054 (42S22): Unknown column 'item_id' in 'field list'
Error in row 4597:
order_id : 1574.0
item_id : 1.0
product_id : 127.0
quantity : 1.0
list_price : 469.99
discount : 0.1
Error: 1054 (42S22): Unknown column 'item_id' in 'field list'
Error in row 4598:
order_id : 1575.0
item_id : 1.0
product_id : 44.0
quantity : 1.0
list_price : 539.99
discount : 0.07
Error: 1054 (42S22): Unknown column 'item_id' in 'field list'
Error in row 4599:
order_id : 1575.0
item_id : 2.0
product_id : 207.0
quantity : 2.0
list_price : 3999.99
discount : 0.1
Error: 1054 (42S22): Unknown column 'item_id' in 'field list'
Error in row 4600:
order_id : 1575.0
item_id : 3.0
product_id : 213.0
quantity : 1.0
list_price : 269.99
discount : 0.07
Error: 1054 (42S22): Unknown column 'item_id' in 'field list'
Error in row 4601:
order_id : 1575.0
item_id : 4.0
product_id : 126.0
quantity : 2.0
list

Error in row 4648:
order_id : 1589.0
item_id : 3.0
product_id : 28.0
quantity : 2.0
list_price : 2499.99
discount : 0.1
Error: 1054 (42S22): Unknown column 'item_id' in 'field list'
Error in row 4649:
order_id : 1590.0
item_id : 1.0
product_id : 175.0
quantity : 2.0
list_price : 3199.99
discount : 0.07
Error: 1054 (42S22): Unknown column 'item_id' in 'field list'
Error in row 4650:
order_id : 1590.0
item_id : 2.0
product_id : 248.0
quantity : 1.0
list_price : 599.99
discount : 0.2
Error: 1054 (42S22): Unknown column 'item_id' in 'field list'
Error in row 4651:
order_id : 1590.0
item_id : 3.0
product_id : 23.0
quantity : 1.0
list_price : 299.99
discount : 0.07
Error: 1054 (42S22): Unknown column 'item_id' in 'field list'
Error in row 4652:
order_id : 1591.0
item_id : 1.0
product_id : 286.0
quantity : 2.0
list_price : 279.99
discount : 0.2
Error: 1054 (42S22): Unknown column 'item_id' in 'field list'
Error in row 4653:
order_id : 1591.0
item_id : 2.0
product_id : 126.0
quantity : 2.0
lis